In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __init__ import *

In [ ]:
nu_fe_sa = pd.read_csv(
    "{}number_feature_sample.tsv".format(PAI),
    sep="\t",
    index_col=0,
)

In [ ]:
le_ = [
    ["CFS: CD4 ({}) - TA ({})", ["S2", "S4", "S6"], ["S1", "S3", "S5"]],
    ["CFS: CD4 ({}) - TA ({})", ["S2"], ["S1"]],
    ["CFS: CD4 ({}) - TA ({})", ["S4"], ["S3"]],
    ["CFS: CD4 ({}) - TA ({})", ["S6"], ["S5"]],
]

ri_ = [
    ["Healthy: CD4 ({}) - TA ({})", ["S8"], ["S7"]],
]

In [ ]:
pa = "{}compare_feature_differences/".format(PAO)

kwat.path.make(pa)

In [ ]:
from numpy import median


def get_difference(nu_fe_sa, sal_, sar_):

    return median(nu_fe_sa.loc[:, sal_].values, axis=1) - median(
        nu_fe_sa.loc[:, sar_].values, axis=1
    )

In [ ]:
for (tel, sall, salr), (ter, sarl, sarr) in itertools.product(le_, ri_):

    dil_ = get_difference(nu_fe_sa, sall, salr)

    dir_ = get_difference(nu_fe_sa, sarl, sarr)

    ch_ = dil_ - dir_

    fe_ = nu_fe_sa.index.values

    nl = tel.format(",".join(sall), ",".join(salr))

    nr = ter.format(",".join(sarl), ",".join(sarr))

    na = "{} - {}".format(nl, nr)

    nac = kwat.path.clean(na)

    pd.Series(data=ch_, index=fe_, name=na).sort_values().to_csv(
        path_or_buf="{}{}.tsv".format(pa, nac), sep="\t"
    )

    an_po_pa = pd.DataFrame(index=fe_)

    an_po_pa.loc[:, nl] = kwat.array.log_with_negative(dil_)

    an_po_pa.loc[:, nr] = kwat.array.log_with_negative(dir_)

    an_po_pa.loc[:, "Size"] = 8

    bo_ = kwat.iterable.check_is_in(
        fe_,
        np.concatenate(
            [fe_[kwat.array.check_is_extreme(ch_, "<>", n_ex=5)], SETTING["fe_"]]
        ),
    )

    an_po_pa.loc[:, "Color"] = np.where(bo_, "#9017e6", "#20d0ba")

    an_po_pa.loc[:, "Opacity"] = np.where(bo_, 1, 0.48)

    an_po_pa.loc[:, "Annotate"] = bo_

    kwat.plot.plot_point(an_po_pa, title=na, pa="{}{}.html".format(pa, nac))